In [ ]:
rm model\ \(1\).zip

In [26]:
!unzip model\ \(2\).zip

Archive:  model (2).zip
replace model/bertmodel.pt/variables/variables.data-00000-of-00001? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace model/bertmodel.pt/variables/variables.index? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [27]:
!pip install -q -U tensorflow-text
!pip install -q tf-models-official

In [ ]:
%ls


 model/  'model (1).zip'   sample_data/


In [39]:
import os
import tensorflow as tf
import pandas as pd
import tensorflow_text

bert_model = tf.saved_model.load('model/bertmodel.pt')



build examples

In [41]:
articles = pd.read_csv('articles_fullcontext_cleaned.csv')
print(articles.head())
examples = articles['content']
print(examples[3])
print(examples)

    domain  ...                                            content
0  cnn.com  ...  Story Highlights • Pelosi accepts gavel as fir...
1  cnn.com  ...  (CNN) -- Barack Obama told supporters that "ch...
2  cnn.com  ...  New Orleans braces for monster hurricane Cresc...
3  cnn.com  ...  Editor's Note: Read live blogging of the Japan...
4  cnn.com  ...  Venice, Louisiana (CNN) -- Gulf coast resident...

[5 rows x 4 columns]
Editor's Note: Read live blogging of the Japan tsunami and earthquake. Are you there? Send your video, pictures to iReport. Tokyo (CNN) -- The morning after Japan was struck by the most powerful earthquake to hit the island nation in recorded history and the tsunami it unleashed -- and even as the earth continued to twitch with aftershocks -- the disaster's massive impact was only beginning to be revealed. Rescue efforts began with the first light as military helicopters plucked survivors from roofs and carried them to safety. The 8.9-magnitude temblor, which was center

Get PRedictions

In [42]:
results = tf.sigmoid(bert_model(tf.constant(examples)))

In [ ]:
print(results)

In [32]:
labels = ['title',"fake", "satire", "bias",  "conspiracy", "junksci", "hate", "clickbait", "unreliable", "political", "reliable"]
full_res = pd.DataFrame(columns=labels)
full_res['title']=articles['title']
# for x in results[:,0]:
#   print(round(x.numpy(),3))
# print([ round(x.numpy(),3) for x in results[:,0]])
# te = [ round(x.numpy(),3) for x in results[:,0]]
# full_res['fake']=te
# full_res['satire']=[round(x.numpy(),3) for x in results[:,1]]
# full_res['bias']=[ round(x.numpy(),3) for x in results[:,2]]
# full_res['conspiracy']=[ round(x.numpy(),3) for x in results[:,3]]
# full_res['junksci']=[ round(x.numpy(),3) for x in results[:,4]]
# full_res['hate']=[ round(x.numpy(),3) for x in results[:,5]]
# full_res['clickbait']=[round(x.numpy(),3)for x in results[:,6] ]
# full_res['unreliable']=[ round(x.numpy(),3) for x in results[:,7]]
# full_res['political']=[round(x.numpy(),3) for x in results[:,8]]
# full_res['reliable']=[round(x.numpy(),3) for x in results[:,9]]
# print(type(full_res['bias'][0]))

full_res['fake']=results[:,0]
full_res['satire']=results[:,1]
full_res['bias']=results[:,2]
full_res['conspiracy']=results[:,3]
full_res['junksci']=results[:,4]
full_res['hate']=results[:,5]
full_res['clickbait']=results[:,6]
full_res['unreliable']=results[:,7]
full_res['political']=results[:,8]
full_res['reliable']=results[:,9]


full_res.round(3)



,title,fake,satire,bias,conspiracy,junksci,hate,clickbait,unreliable,political,reliable
0,Pelosi: 'Let us work,0.093,0.998,0.902,0.889,0.008,0.009,0.995,0.259,0.029,0.435
1,Obama: 'This is your,0.970,0.990,0.979,0.233,0.006,0.095,0.937,0.023,0.002,0.968
2,New Orleans braces for monster,0.015,0.600,0.874,0.696,0.001,0.771,0.766,0.978,0.869,0.033
3,"Widespread destruction from Japan earthquake,",0.961,0.786,0.996,0.177,0.414,0.692,0.672,0.506,0.003,0.060
4,"As oil spill nears Gulf Coast, experts issue dire",0.387,0.999,0.974,0.062,0.009,0.028,0.999,0.050,0.006,0.615
...,...,...,...,...,...,...,...,...,...,...,...
78,Obama Sweeps to Historic,0.507,0.991,0.544,0.012,0.830,0.088,0.995,0.265,0.005,0.981
79,Last Call in,0.736,0.771,0.399,0.967,0.124,0.002,0.913,0.794,0.093,0.995
80,Deadly Blasts Rock,0.667,0.930,0.922,0.061,0.922,0.127,0.934,0.073,0.020,0.959
81,"Ukraine Accuses Russia of Invasion, West to Co...",0.570,0.798,0.190,0.104,0.592,0.059,0.999,0.358,0.006,0.999


In [33]:
def get_classification(scores, precision):
  classes=["fake", "satire", "bias",  "conspiracy", "junksci", "hate", "clickbait", "unreliable", "political", "reliable"]
  class_str = ""
  
  for i in range(len(scores)):
    
    if scores[i] > precision:
      class_str += classes[i]
      class_str += '|'
  return class_str

In [45]:
def get_single_classification(scores):
  classes=["fake", "satire", "bias",  "conspiracy", "junksci", "hate", "clickbait", "unreliable", "political", "reliable"]
  max_index = 0
  for i in range(1,len(scores)):
    if scores[i] > scores[max_index]:
      max_index=i
  return classes[max_index]
      


In [61]:
precision = .9
classifications = []
for res in results:
  classifications.append(get_classification(res,precision))
  # classifications.append(get_single_classification(res))
full_res['classification']=classifications
full_res['source']=articles['domain']
full_res['topic']=articles['story_label']

# print(full_res[['classification','source']])
%load_ext google.colab.data_table
full_res[['title','topic','classification','source']]

The google.colab.data_table extension is already loaded. To reload it, use:
  %reload_ext google.colab.data_table


,title,topic,classification,source
0,Pelosi: 'Let us work,House Speaker,satire|bias|clickbait|,cnn.com
1,Obama: 'This is your,Obama Elected,fake|satire|bias|clickbait|reliable|,cnn.com
2,New Orleans braces for monster,Hurricane Katrina,unreliable|,cnn.com
3,"Widespread destruction from Japan earthquake,",Japan Earthquake & Tsunamis,fake|bias|,cnn.com
4,"As oil spill nears Gulf Coast, experts issue dire",BP Oil Spill,satire|bias|clickbait|,cnn.com
...,...,...,...,...
78,Obama Sweeps to Historic,Obama Elected,satire|clickbait|reliable|,wsj.com
79,Last Call in,London Olympics,conspiracy|clickbait|reliable|,wsj.com
80,Deadly Blasts Rock,Boston Marathon Bombings,satire|bias|junksci|clickbait|reliable|,wsj.com
81,"Ukraine Accuses Russia of Invasion, West to Co...",Russia Invades Ukraine,clickbait|reliable|,wsj.com


In [62]:
f = open('multi_class.csv', 'w')
full_res.to_csv(f)
f.close()
print(csv)



,title,fake,satire,bias,conspiracy,junksci,hate,clickbait,unreliable,political,reliable,classification,source,topic
0,Pelosi: 'Let us work,0.09287065,0.99815226,0.9022277,0.8887222,0.007603079,0.00897184,0.99539876,0.25883812,0.029197007,0.43471688,satire,cnn.com,House Speaker
1,Obama: 'This is your,0.9697237,0.98960775,0.9787611,0.23330167,0.005815059,0.09516144,0.9374057,0.022784919,0.0022981763,0.9675516,satire,cnn.com,Obama Elected
2,New Orleans braces for monster,0.014797628,0.60046494,0.87362325,0.69559914,0.00089374185,0.7714841,0.7662373,0.9783585,0.86934245,0.032830447,unreliable,cnn.com,Hurricane Katrina
3,"Widespread destruction from Japan earthquake,",0.9608929,0.78566206,0.99589694,0.17716941,0.4137558,0.6918177,0.6715974,0.50562686,0.0027413368,0.060318857,bias,cnn.com,Japan Earthquake & Tsunamis
4,"As oil spill nears Gulf Coast, experts issue dire",0.3866,0.999349,0.9737259,0.06243369,0.008873194,0.028157175,0.99885345,0.049531847,0.0061636567,0.6150416,satire,cnn.com,BP

counts by source

In [57]:
def get_counts(cnn, fox, nbc, nyp, wsg, classifications, sour):
  classes=["fake", "satire", "bias",  "conspiracy", "junksci", "hate", "clickbait", "unreliable", "political", "reliable"]
  # cl = classifications.split('|')[:-1]
  cl = classifications
  i = cl
  # for i in cl:
    # print(i)
    # print(sour)
  if sour == "cnn.com":
    cnn[classes.index(i)] += 1
  elif sour == 'foxnews.com':
    fox[classes.index(i)]+=1
  elif sour == "nbcnews.com":
    nbc[classes.index(i)]+=1
  elif sour == "nypost.com":
    nyp[classes.index(i)]+=1
  elif sour == "wsj.com":
    wsj[classes.index(i)]+=1
  



In [58]:
classes=["fake", "satire", "bias",  "conspiracy", "junksci", "hate", "clickbait", "unreliable", "political", "reliable"]


counts = pd.DataFrame(columns = classes)
domains = ["cnn.com", "	nbcnews.com", "wsj.com", "nypost.com", "foxnews.com"]

# counts['domain'] = ["cnn.com", "foxnews.com", "	nbcnews.com", "nypost.com", "wsj.com"]



# counts = classes
cnn=[0,0,0,0,0,0,0,0,0,0]
fox=[0,0,0,0,0,0,0,0,0,0]
nbc=[0,0,0,0,0,0,0,0,0,0]
nyp=[0,0,0,0,0,0,0,0,0,0]
wsj=[0,0,0,0,0,0,0,0,0,0]

full_res.apply(lambda x: get_counts(cnn, fox, nbc, nyp, wsj, x['classification'], x['source']), axis=1)
# full_res.apply(lambda x: print(x['classification']), axis=1)
# print(cnn)
counts.loc[0] = cnn
counts.loc[2] = nbc
counts.loc[4] = wsj
counts.loc[3] = nyp
counts.loc[1] = fox




counts.insert(0, 'source', value=domains)
print(f"Number of Articles of each Class by Domain, Threshold {precision}")
counts




Number of Articles of each Class by Domain, Threshold 0.9


,source,fake,satire,bias,conspiracy,junksci,hate,clickbait,unreliable,political,reliable
0,cnn.com,0,8,2,0,1,0,3,2,0,1
2,\tnbcnews.com,1,3,0,0,5,0,2,3,0,2
4,wsj.com,0,3,0,0,3,0,5,2,1,3
3,nypost.com,5,0,1,0,1,0,3,1,3,3
1,foxnews.com,2,2,0,1,3,2,5,0,0,1


In [60]:
f = open('single_counts.csv', 'w')
counts.to_csv(f)
print(counts.to_csv())

,source,fake,satire,bias,conspiracy,junksci,hate,clickbait,unreliable,political,reliable
0,cnn.com,0,8,2,0,1,0,3,2,0,1
2,	nbcnews.com,1,3,0,0,5,0,2,3,0,2
4,wsj.com,0,3,0,0,3,0,5,2,1,3
3,nypost.com,5,0,1,0,1,0,3,1,3,3
1,foxnews.com,2,2,0,1,3,2,5,0,0,1



In [37]:
full_res.loc[29]

title             'Storm Area 51' raid brings alien 'community o...
fake                                                       0.282041
satire                                                     0.913391
bias                                                       0.937005
conspiracy                                                0.0488398
junksci                                                    0.710416
hate                                                        0.99812
clickbait                                                  0.427085
unreliable                                                0.0413512
political                                                  0.980749
reliable                                                  0.0589758
classification                          satire|bias|hate|political|
source                                                  foxnews.com
topic                                                  Area 51 Raid
Name: 29, dtype: object

In [38]:
full_res['topic'] = articles['story_label']
ful_res

NameError: ignored

In [ ]:
print(articles.loc[19]['content'])
print(articles.loc[29]['content'])
print(articles.loc[21]['content'])

More than 3,000 people a day have a heart attack. If you're one of them the day after your H1N1 flu shot, will you worry the vaccine was to blame and not the more likely culprit, all those burgers and fries? The government is starting an unprecedented system to track possible side effects as mass flu vaccinations begin next month. The idea is to detect any rare but real problems quickly, and explain the inevitable coincidences that are sure to cause some false alarms. "Every day, bad things happen to people. When you vaccinate a lot of people in a short period of time, some of those things are going to happen to some people by chance alone," said Dr. Daniel Salmon, a vaccine safety specialist at the Department of Health and Human Services. Health authorities hope to vaccinate well over
From the bright lights and bustling streets of downtown Las Vegas' famous Fremont Street to the dusty and dry remote Nevada high-desert, "Storm Area 51" hype has brought out the true believers in-force w